In [9]:
from MCTS import MCTS

from sogo.SogoGame import SogoGame, display as display_board
import numpy as np
from sogo.keras.NNet import NNetWrapper as NNet
from Timer import Timer

from Config import Config
from sogo.keras.NNet import NNArgs
# nnet players
config = Config(
    load_folder_file=('./save/', 'mcts1024_eps40_iter17.h5'),
    num_mcts_sims=150,
    root_dirichlet_alpha=0.3,
    root_exploration_fraction=0.0,
    mcts_discount=0.9,
    pb_c_base=19652,
    pb_c_init=1.25)
config.nnet_args = NNArgs(lr=0.001, 
                              batch_size=1024, 
                              epochs=20)

game = SogoGame(4)

def setup_board(plays,verbose=True): 
    board = game.init_board()
    player = 1    
    for play in plays:
        board, player = game.next_state(board, player,play)
    if verbose:
        display_board(board)
    return board, player

def display_probs(pi, prefix="Probs"):
        print(f"{prefix}: {np.array2string(np.array(pi), precision=2, separator=',', suppress_small=True, max_line_width=200)}")

In [10]:
from NeuralNet import NeuralNet
from Game import Game

class NN(NeuralNet):
  def __init__(self,game:Game):
    self.game = game
  def predict(self, board):
    return np.ones(self.game.action_size())/self.game.action_size(), 0


dummy_nn = NN(game)
dummy_mcts = MCTS(game, dummy_nn, config)

def mcts_only_pred(plays, root=None, verbose=False):
    b,p = setup_board(plays, verbose = verbose)
    with Timer("MCTS only prediction"):
        pi, root = dummy_mcts.get_action_prob(b, p, root)
    display_probs(pi)
    return root          

In [11]:
nn = NNet(game, config)
nn.load_checkpoint(*(config.load_folder_file))

def nn_pred(plays, verbose=False):
    b,p = setup_board(plays, verbose = verbose)
    b = game.canonical_board(b,p)
    with Timer("NN prediction"):
        pi, v = nn.predict(b)
    display_probs(pi,"NNet")
    
mcts = MCTS(game, nn, config)

def mcts_player(x, player):
    canonical_board = game.canonical_board(x, player)
    return np.argmax(pi), root

def test_mcts(plays, expected, verbose=False):
    board, player = setup_board(plays, verbose = verbose)    
    with Timer() as t:
        pi, root = mcts.get_action_prob(board,player)
    play = np.argmax(pi)
    new_board, new_player = game.next_state(board, player, play)
    if verbose:
        display_board(new_board)    
    nn_pred(plays)
    display_probs(pi, "MCTS")
    valid = play in expected if isinstance(expected,list) else play == expected
    print(f"MCTS made {'valid' if valid else 'bad' } play: {play} (expected: {expected}) in  {t.interval:0.3f} sec")
    return root
          
def mcts_pred(plays, root=None, verbose=False):
    b,p = setup_board(plays, verbose = verbose)
    with Timer("MCTS prediction"):
        pi, root = mcts.get_action_prob(b, p, root)
    display_probs(pi,"MCTS")
    return root

          

In [12]:
def pref_path(r):
    if len(r.children) == 0:
        return []
    _, a, c = max((c.visit_count, a, c) for a,c in r.children.items())
    return [a] + pref_path(c)

### Play with sure win, doesn't favor fast win.

In [13]:
pl = [x['i'] + 4*x['j'] for x in [{'i': 3, 'j': 0}, {'i': 0, 'j': 3}, {'i': 2, 'j': 0}, {'i': 2, 'j': 0},   {'i': 2, 'j': 0},   {'i': 1, 'j': 2},   {'i': 2, 'j': 0},   {'i': 0, 'j': 0},   {'i': 3, 'j': 0},   {'i': 3, 'j': 3},   {'i': 3, 'j': 1},   {'i': 3, 'j': 1},   {'i': 3, 'j': 1},   {'i': 3, 'j': 0},   {'i': 3, 'j': 0},   {'i': 2, 'j': 1},   {'i': 3, 'j': 1},   {'i': 3, 'j': 2},   {'i': 3, 'j': 2},   {'i': 3, 'j': 2},   {'i': 3, 'j': 3},   {'i': 2, 'j': 1},   {'i': 3, 'j': 3},   {'i': 3, 'j': 3},   {'i': 3, 'j': 2},   {'i': 0, 'j': 2},   {'i': 1, 'j': 3}]]

In [14]:
setup_board(pl)
nn_pred(pl)
r = mcts_pred(pl)

z3+--------+
3 |- - - O |
2 |- - - X |
1 |- - - X |
0 |- - X X |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - X |
2 |- - - O |
1 |- - - X |
0 |- - X O |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - X |
2 |- - - X |
1 |- - O O |
0 |- - O X |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X - O |
2 |O O - O |
1 |- - O X |
0 |O - X X |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.534 sec
NNet: [0.01,0.03,0.  ,0.  ,0.12,0.02,0.11,0.  ,0.3 ,0.03,0.07,0.07,0.03,0.04,0.18,0.  ]
MCTS prediction took 0.819 sec
MCTS: [0.  ,0.  ,0.  ,0.  ,0.3 ,0.  ,0.11,0.  ,0.22,0.  ,0.18,0.  ,0.  ,0.04,0.15,0.  ]


In [15]:
mcts_only_pred(pl)

MCTS only prediction took 0.199 sec
Probs: [0.01,0.01,0.  ,0.  ,0.44,0.01,0.01,0.  ,0.01,0.01,0.44,0.  ,0.01,0.01,0.01,0.  ]


In [16]:
r.print(3)

 -> v:-0.81 n:150 p:1.0 tp:1
  4 -> v:1.0 n:44 p:0.13 tp:-1
  6 -> v:0.88 n:16 p:0.12 tp:-1
    0 -> v:-1.0 n:1 p:0.014 tp:1
    4 -> v:-0.92 n:6 p:0.62 tp:1
    5 -> v:-1.0 n:1 p:0.0098 tp:1
    6 -> v:-1.0 n:1 p:0.11 tp:1
    8 -> v:-1.0 n:1 p:0.07 tp:1
    9 -> v:-1.0 n:1 p:0.057 tp:1
    10 -> v:-1.0 n:1 p:0.07 tp:1
    12 -> v:-1.0 n:1 p:0.0074 tp:1
    13 -> v:-1.0 n:1 p:0.022 tp:1
    14 -> v:-1.0 n:1 p:0.021 tp:1
  8 -> v:0.79 n:33 p:0.32 tp:-1
    0 -> v:-1.0 n:1 p:0.018 tp:1
    1 -> v:-1.0 n:1 p:0.0043 tp:1
    4 -> v:-0.82 n:13 p:0.35 tp:1
    5 -> v:-1.0 n:1 p:0.009 tp:1
    6 -> v:-0.82 n:6 p:0.17 tp:1
    8 -> v:-0.88 n:4 p:0.15 tp:1
    9 -> v:-1.0 n:1 p:0.074 tp:1
    10 -> v:-0.95 n:2 p:0.1 tp:1
    12 -> v:-1.0 n:1 p:0.015 tp:1
    13 -> v:-1.0 n:1 p:0.034 tp:1
    14 -> v:-1.0 n:1 p:0.074 tp:1
  10 -> v:1.0 n:27 p:0.08 tp:-1
  13 -> v:0.9 n:6 p:0.04 tp:-1
    4 -> v:-0.95 n:2 p:0.5 tp:1
    6 -> v:-1.0 n:1 p:0.085 tp:1
    8 -> v:-1.0 n:1 p:0.12 tp:1
    10 -> v:-1.

### State requires defense against 2 step win as player 2

In [43]:
play, valid = [0,7,3,11,5,15,13], [1,2,9]
test_mcts(play, valid);

NN prediction took 0.022 sec
NNet: [0.1 ,0.29,0.05,0.27,0.  ,0.  ,0.01,0.01,0.03,0.19,0.01,0.  ,0.01,0.02,0.  ,0.  ]
MCTS: [0.35,0.41,0.02,0.12,0.  ,0.  ,0.  ,0.  ,0.  ,0.1 ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ]
MCTS made valid play: 1 (expected: [1, 2, 9]) in  3.281 sec


In [10]:
config.num_mcts_sims = 40000
play = [0,7,3,11,5,15,13]

r1 = mcts_only_pred(play) # differs from test_mcts, direct canonical vs this
rc = mcts_c_only_pred(play)

MCTS only prediction took 97.231 sec
Probs: [0.05,0.24,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.09,0.05,0.05,0.05,0.05,0.05,0.05]
MCTS canonical only prediction took 95.508 sec
Probs: [0.05,0.24,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.09,0.05,0.05,0.05,0.05,0.05,0.05]


### Required defense against 1 step win

In [9]:
config.num_mcts_sims = 1000
play, valid  = [0,1,0,1,0], 1
nn_pred(play)
mcts_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O X - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O X - - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.699 sec
Probs: [0.9 ,0.  ,0.  ,0.01,0.  ,0.01,0.01,0.  ,0.01,0.  ,0.01,0.01,0.  ,0.  ,0.02,0.01] Value: -0.64
MCTS prediction took 22.706 sec
Probs: [0.95,0.02,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ,0.  ]
MCTS only prediction took 2.476 sec
Probs: [0.68,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02]


In [10]:
config.num_mcts_sims = 100
play = [2,1,2,1,2]
nn_pred(play)
mcts_pred(play)
mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.040 sec
Probs: [0.01,0.  ,0.91,0.01,0.01,0.01,0.  ,0.  ,0.  ,0.01,0.  ,0.01,0.01,0.01,0.  ,0.01] Value: 0.81
MCTS prediction took 2.208 sec
Probs: [0.01,0.  ,0.85,0.02,0.02,0.01,0.01,0.  ,0.  ,0.02,0.  ,0.01,0.02,0.02,0.  ,0.01]
MCTS only prediction took 0.215 sec
Probs: [0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07]


In [18]:
config.num_mcts_sims = 100
play = [2,1,2,1,2]
nn_pred(play)
r1 = mcts_pred(play)
r0 = mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.013 sec
Probs: [0.04,0.05,0.44,0.02,0.04,0.04,0.04,0.02,0.02,0.04,0.05,0.04,0.03,0.05,0.05,0.04] Value: 0.90
MCTS prediction took 1.370 sec
Probs: [0.01,0.15,0.41,0.01,0.05,0.01,0.01,0.01,0.03,0.04,0.11,0.03,0.01,0.05,0.01,0.06]
MCTS only prediction took 0.109 sec
Probs: [0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.07,0.07,0.07,0.07]


In [20]:
config.num_mcts_sims = 100
play = [2,1,2,1,2,2]
nn_pred(play)
mcts_pred(play)
mcts_only_pred(play)

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - X - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - O - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- X O - |
z0+--------+
   0 1 2 3 
--
NN prediction took 0.013 sec
Probs: [0.06,0.06,0.  ,0.07,0.06,0.11,0.08,0.06,0.05,0.06,0.06,0.05,0.08,0.04,0.11,0.06] Value: -0.27
MCTS prediction took 1.953 sec
Probs: [0.22,0.02,0.  ,0.02,0.02,0.07,0.09,0.06,0.04,0.02,0.09,0.01,0.02,0.01,0.29,0.02]
MCTS only prediction took 0.111 sec
Probs: [0.06,0.06,0.  ,0.06,0.06,0.06,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07,0.07]


### States that requires defense against 1 step win

In [20]:
test_mcts([0,8,0,8,0],0);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |X - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |O - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |- - - - |
2 |X - - - |
1 |- - - - |
0 |O - - - |
z0+--------+
   0 1 2 3 
--
MCTS made correct play in  11.332 sec


### State requires defense against 2 step win as player 1

In [22]:
test_mcts([12,0,7,3,11,5,15,13],1);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X - O |
2 |- - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X - O |
2 |- O - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
MCTS made incorrect play in  11.554 sec


### State with 2 step win as player 1

In [24]:
test_mcts([0,7,3,11,5,15,13,12],1);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |X O - X |
2 |- - - X |
1 |- O - X |
0 |O - - O |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |X O - X |
2 |- - - X |
1 |- O - X |
0 |O O - O |
z0+--------+
   0 1 2 3 
--
MCTS made correct play in  5.456 sec


### State with 2 step win as player 2

In [23]:
test_mcts([14,0,7,3,11,5,15,13,12],1);

z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X O O |
2 |- - - O |
1 |- X - O |
0 |X - - X |
z0+--------+
   0 1 2 3 
--
z3+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z3+--------+
   0 1 2 3 
z2+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z2+--------+
   0 1 2 3 
z1+--------+
3 |- - - - |
2 |- - - - |
1 |- - - - |
0 |- - - - |
z1+--------+
   0 1 2 3 
z0+--------+
3 |O X O O |
2 |- - - O |
1 |- X - O |
0 |X X - X |
z0+--------+
   0 1 2 3 
--
MCTS made correct play in  5.127 sec


#### Other Stuff